<a href="https://colab.research.google.com/github/Sapienza-AI-Lab/esercitazione6-22-23/blob/main/Exercise2.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

# Esercizio 2
In questo esercizio utilizzerete il dataset Heart Disease per costruire un modello in grado di predire se un paziente ha o meno una malattia cardiaca. Il dataset è composto da 303 pazienti, ognuno dei quali è descritto da 13 attributi. L'attributo target è un valore intero che va da 0 (assenza) a 4.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, StratifiedKFold, learning_curve, train_test_split
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

### Caricamento del dataset

Attributes:
* Age
* Sex:
    * Male -> 1
    * Female -> 0
* Chest Pain Type: 
    * angina -> 1
    * abnang -> 2
    * notang -> 3
    * asympt -> 4
* Resting Blood Pressure
* Cholesterol
* Fasting Blood Sugar:
    * < 120 -> 1
    * \>= 120 -> 0
* Resting ECG:
    * norm -> 0
    * abn -> 1
    * hyper -> 2
* Max Heart Rate
* Exercise Induced Angina:
    * yes -> 1
    * no -> 0
* Oldpeak
* Slope:
    * up -> 1
    * flat -> 2
    * down -> 3
* Number of vessels colored
* Thal:
    * norm -> 3
    * fixed -> 6
    * rever -> 7
* Target:
    * 0 -> no disease
    * 1,2,3,4 -> disease



In [ ]:
cols = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','target']
xcols = cols[0:-1]
ycol = cols[-1]
df = pd.read_csv('data/processed.cleveland.data', header=None, index_col=None, names=cols, na_values=['?'])
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df = df.dropna()

In [ ]:
X = df[xcols]
m, n = X.shape

# Binarize the target
y = (df[ycol] > 0).astype(int)

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Normalize the data
normalize = True

if normalize:
    xmean = X_train.mean()
    xstd = X_train.std()
    # Note that we use the mean and std of the training set to normalize both the training and test sets
    X_train = (X_train - xmean)/xstd
    X_test = (X_test - xmean)/xstd

### Define the model

In [ ]:
# Define logistic regression model
model = LogisticRegression(solver='lbfgs', n_jobs=-1)

### Grid Search

In [ ]:
# Define scorer
scorer = make_scorer(accuracy_score)

# Define grid search parameters
grid = dict()
grid['C'] = np.logspace(-6, 3, 10)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scorer, error_score=0)

In [ ]:
# Perform grid search
start = time.time()
grid_result = grid_search.fit(X_train, y_train)
end = time.time()
print('Grid search time =', end - start)

In [ ]:
# Summarize results
print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('%f (%f) with: %r' % (mean, stdev, param))

In [ ]:
# Plot grid search results
plt.errorbar(grid['C'], means, yerr=stds)
plt.xscale('log')
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
# Define logistic regression model with best parameters
model = LogisticRegression(solver='lbfgs', n_jobs=-1, C=grid_result.best_params_['C'])

### Learning Curves

In [ ]:
# Plot learning curve
scorer = make_scorer(accuracy_score)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
train_sizes, train_scores, test_scores = learning_curve(
    model, X_train, y_train, cv=cv, scoring=scorer, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10))
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='training accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean, color='green', linestyle='--', marker='s', markersize=5, label='validation accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.6, 1.0])
plt.show()

### Train

In [ ]:
# Fit model on all training data
model.fit(X_train, y_train)

### Test

In [ ]:
# Predict on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Test accuracy =', accuracy)

### Plot model coefficients

In [ ]:
# Plot model coefficients
coeffs = pd.DataFrame(model.coef_, columns=xcols)
coeffs = coeffs.transpose()
coeffs.columns = ['Coefficients']
coeffs = coeffs.sort_values(by='Coefficients', ascending=False)
sns.barplot(x=coeffs['Coefficients'], y=coeffs.index)
plt.show()


In [ ]:
# Plot model coefficients without using only matplotlib
plt.bar(range(n), model.coef_[0])
plt.xticks(range(n), xcols, rotation=90)
plt.show()

# Task: Ripetete l'analisi e l'addestramento sul dataset YearPredictionMSD:

Il dataset Year Prediction MSD si puà scaricare da questo link: https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD

![YPMSD](data/ypmsd.jpg)
- **Nota 1:** Il dataset è molto grande. Per testare la procedura di analisi e la correttezza del codice, prima
provate su un sottoinsieme dei dati.
- **Nota 2:** Il problema può essere trattato come un problema di classificazione o di regressione. Voi iniziate a risolverlo come classificazione, poi, se volete, potete provare a risolverlo come regressione.
